# EmbedX: Multi-lingual Embedding Model

## Project Overview

- EmbedX is an Natural Language Processing project focused on developing and comparing different word embedding approaches across multiple languages. 
- The project aims to evaluate the effectiveness of various embedding techniques through practical applications in chatbot development and comprehensive comparative analysis.

## Key Components of this project

- **Part 1.** Multi-lingual Word2Vec Implementation
    - Focus on creating a model to produce Word2Vec embeddings for non-English languages.

- **Part 2.** RNN-based Embedding Architecture 
    - Development of a custom RNN architecture for embedding generation. 
    - Comparative analysis against standard Keras Word2Vec embeddings.

- **Part 3.** Chatbot Integration & Evaluation 
    - Development of an embedding-powered chatbot system.
    - Comparative testing between English and non-English embedding models
    - Performance benchmarking across different embedding sources


- **Part 4.** Embedding Quality Assessment
    - Comparative analysis of Word2Vec and RNN-based approaches
    - Implementation of multiple evaluation metrics


- **Part 5.** DNN Comparative Study
    - Implementation of a shallow DNN for embedding generation
    - Head-to-head comparison with RNN-based embeddings

## Priority Focus
- The core focus is on developing and optimizing the RNN-based embedding architecture, as this forms the foundation for subsequent comparative analyses and applications.

---

## Part 2. RNN-based Embedding Architecture 
- Development of a custom RNN architecture for embedding generation. 
- Comparative analysis against standard Keras Word2Vec embeddings.

### Approach

- Import necessary libraries

In [1]:
import copy
import pandas as pd
import numpy as np
from string import digits
import pickle

import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from indicnlp.tokenize import indic_tokenize

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model, load_model

- Preprocess the text from the file \
Process the file line by line, remove unwanted characters, and breakdown the content into sentences based on Hindi punctuation

In [2]:
hi_punct = ['!', '?', '|', '||']
sentences = []
with open("dataset/Panchtantra.txt", 'r', encoding="utf-8") as file_data: 
    
    sentence = ''
    paragraph_p = 0
    for l,line in enumerate(file_data): 
        
        # Remove unwanted characters
        line = line.strip() \
                    .replace('।', '') \
                    .replace('॥', '') \
                    .replace('\ufeff', '') \
                    .replace('\u200C', '') \
                    .replace('“', '') \
                    .replace('”', '') \
                    .replace('‘', '') \
                    .replace('’', '') \
                    .replace('—', '') \
                    .replace('-', '') \
                    .replace('$', '')
                    
        
        # One \n is just a carriage return. Two \n's means 
        # a new paragraph, start a new sentence
        if len(line) == 0:
            #print('empty line!')
            paragraph_p += 1
            if paragraph_p == 2:
                if 0 < len(sentence):
                    sentences.append(sentence)
                    sentence = ''
                paragraph_p = 0
            continue
        
        # detect sentence delimeters
        delimeters = []
        for c in hi_punct:
            line_copy = copy.copy(line)
            while -1 < line_copy.find(c):
                d = line_copy.find(c)
                delimeters.append(d)
                line_copy = line_copy[:d] + '_' + line_copy[d+1:]
                    
        delimeters.sort()

            
        # no sentence delimeters: keep appending
        if len(delimeters) == 0:
            sentence += line
            
        # detected sentence delimeter(s), terminate sentence
        else:
                
            last_index = 0
            for i in delimeters:
                sentence += line[last_index : i]
                sentences.append(sentence)

                sentence = ''
                last_index = i + 1
                    
            # at the end of the indexes traversal, add remaining words to next sentence     
            if last_index <= len(line):
                dangling = line[last_index :].strip()
                if 0 < len(dangling):
                    sentence = dangling
                    #print("seeding:", sentence)
                else:
                    sentence = ''
            else:
                sentence = ''


- Check the sentences list

In [3]:
[(i, s) for i,s in enumerate(sentences)]

[(0,
  'पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहानियों का संग्रह है जिसके रचयिता आचार्य विष्णु शर्मा है पंचतंत्र की कहानी में बच्चोंके साथसाथ बड़े भी रुचि लेते हैं पंचतंत्र की कहानी के पीछे कोई ना कोई शिक्षा या मूल छिपा होता है जो हमें सीखदेती है पंचतंत्र की कहानी बच्चे बड़ी चाव से पढ़ते हैं तथा सीख लेते हैं पंचतंत्र की कछ कहानियां ऐसी भी है जोहिंदी में कहानी लेखन में दी जाती हैं तथा इसके साथसाथ कई परीक्षाओं में भी 80780 6 (9 कुछपाठयपृस्तक में दी होती है जो लिखने को आती हैं महत्वपूर्ण और आकर्षक 40 कहानियां इस ब्लॉग में दी गई हैं'),
 (1,
  'आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा पंचतंत्र संस्कत की नीति पर्तक के लेखक माने जाते हैं जब यह ग्रंथ बनकर तैयारहुआ तब विष्ण शर्मा की उम्र 40 सात्र थी विष्ण शर्मा दक्षिण भारत के महिलारोप्य नामक नगर में रहते थे एकराजा के 3 मर्ख पत्र थे जिनकी जिम्मेदारी विष्ण शर्मा को दे दी गई विष्ण शर्मा को यह पता था कि यह इतने मूर्ख हैंकि इनको प्राने तरीकों से नहीं पढ़ाया जा सकता तब उन्होंने जंत कथाओं के द्वारा पढ़ाने का निश्चय कियापंचतंत्र को पांच समूह म

- Preview the first 5 sentences

In [4]:
sentences[:5]

['पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहानियों का संग्रह है जिसके रचयिता आचार्य विष्णु शर्मा है पंचतंत्र की कहानी में बच्चोंके साथसाथ बड़े भी रुचि लेते हैं पंचतंत्र की कहानी के पीछे कोई ना कोई शिक्षा या मूल छिपा होता है जो हमें सीखदेती है पंचतंत्र की कहानी बच्चे बड़ी चाव से पढ़ते हैं तथा सीख लेते हैं पंचतंत्र की कछ कहानियां ऐसी भी है जोहिंदी में कहानी लेखन में दी जाती हैं तथा इसके साथसाथ कई परीक्षाओं में भी 80780 6 (9 कुछपाठयपृस्तक में दी होती है जो लिखने को आती हैं महत्वपूर्ण और आकर्षक 40 कहानियां इस ब्लॉग में दी गई हैं',
 'आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा पंचतंत्र संस्कत की नीति पर्तक के लेखक माने जाते हैं जब यह ग्रंथ बनकर तैयारहुआ तब विष्ण शर्मा की उम्र 40 सात्र थी विष्ण शर्मा दक्षिण भारत के महिलारोप्य नामक नगर में रहते थे एकराजा के 3 मर्ख पत्र थे जिनकी जिम्मेदारी विष्ण शर्मा को दे दी गई विष्ण शर्मा को यह पता था कि यह इतने मूर्ख हैंकि इनको प्राने तरीकों से नहीं पढ़ाया जा सकता तब उन्होंने जंत कथाओं के द्वारा पढ़ाने का निश्चय कियापंचतंत्र को पांच समूह में बनाया गया 

- Convert sentences to a DataFrame for applying lambda functions

In [5]:
sentences_p = pd.DataFrame(sentences)
sentences_p

,0
0,"पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान..."
1,आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2,चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3,"तो नहीं, उसका खून चूसने के लिए"
4,""" खेंटमल बोला, ""लेकिन मैं तम्हारा मेहमान है, म..."
...,...
81,सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82,सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83,शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84,अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...


- Add spaces around punctuation marks for each sentence

In [6]:
sentences_p[0] = sentences_p[0].apply(lambda x: re.sub(r"([!#$%&\()*+,-./:;<=>?@[\\]^_`{|}~])", r" \1 ", x))

In [7]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4     " खेंटमल बोला, "लेकिन मैं तम्हारा मेहमान है, म...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Remove digits from the sentences

In [8]:
remove_digits = str.maketrans('', '', digits)
sentences_p[0] = sentences_p[0].apply(lambda x: x.translate(remove_digits))

In [9]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4     " खेंटमल बोला, "लेकिन मैं तम्हारा मेहमान है, म...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Remove single quotes from the sentences

In [10]:
# Remove quotes, saveguard commas:
sentences_p[0] = sentences_p[0].apply(lambda x: re.sub("'", '', x))

In [11]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4     " खेंटमल बोला, "लेकिन मैं तम्हारा मेहमान है, म...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Remove double quotes from the sentences

In [12]:
sentences_p[0] = sentences_p[0].apply(lambda x: re.sub('"', '', x))

In [13]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4      खेंटमल बोला, लेकिन मैं तम्हारा मेहमान है, मझे...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Tokenize sentences into words using Indic NLP tokenizer \
Separate each token with a space

In [14]:
sentences_p[0] = sentences_p[0].apply(lambda x: " ".join(indic_tokenize.trivial_tokenize(x)))

In [15]:
len(sentences_p)
sentences_p[0][:1000]

0     पंचतंत्र की कहानियांपंचतंत्र नीति , कथा और कहा...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                       तो नहीं , उसका खून चूसने के लिए
4     खेंटमल बोला , लेकिन मैं तम्हारा मेहमान है , मझ...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Preprocess each sentence with `<start>` and `<end>` tokens

In [16]:
def preprocess_sentence(w):
    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

- Create a dataset from preprocessed sentences

In [17]:
def preprocess_sentence_no_tags(w):
    w = w.strip()

    return w

In [18]:
def create_dataset(start, end):
    #sentence_pairs = [[preprocess_sentence_no_tags(l[0]), preprocess_sentence(l[1])] for l in lines[start:end].values]
    sentences_preprocessed = [preprocess_sentence(l) for l in sentences_p[0][start:end]]
    #return zip(*sentence_pairs)
    return sentences_preprocessed

In [19]:
def create_full_dataset():
    #sentence_pairs = [[preprocess_sentence_no_tags(l[0]), preprocess_sentence(l[1])] for l in lines.values]
    sentences_preprocessed = [preprocess_sentence(l) for l in sentences_p[0]]
    return sentences_preprocessed

- Tokenize the sentences into sequences of integers using TensorFlow's `Tokenizer` \ 
- Ensure the sequences are then padded to maintain uniform length

In [20]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

- Load the dataset

In [21]:
def load_dataset(start, end):
    # creating cleaned input, output pairs
    #inp_lang, targ_lang = create_dataset(start, end)
    inp_lang = create_dataset(start, end)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)

    return input_tensor, inp_lang_tokenizer

In [22]:
def load_full_dataset():
    # creating cleaned input, output pairs
    inp_lang = create_full_dataset()
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)

    return input_tensor, inp_lang_tokenizer

- Preview a sentence from the dataset

In [23]:
zh = create_full_dataset()
print(zh[-1])

<start> अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने पैर पर चल रहे थे कोई भी अपने मम्मी पापा के गोदी में नहीं चल रहा थाइस पर अमन भी अपने छोटेछोटे पैरों से चलने लगा इस पर अमन के मम्मी पापा को बहुत खुशी हुई क्योंकिअब उसका बेटा चलना सीख रहा था <end>


- Load a subset of the dataset (sentences from index 1 to 40), and tokenize them

In [24]:
num_examples_p = 40 
input_tensor_p, inp_lang_p  = load_dataset(1, 40)

# Calculate max_length of the target tensors
max_length_inp_p = input_tensor_p.shape[1]
max_length_inp_p

137

- Creating training and validation sets using an 90-10 split

In [25]:
input_tensor_train_p, input_tensor_val_p = train_test_split(input_tensor_p, test_size=0.1)

# Show length
len(input_tensor_train_p), len(input_tensor_val_p)

(35, 4)

In [26]:
num_examples = len(sentences_p[0])
input_tensor, inp_lang = load_full_dataset()

# Calculate max_length of the target tensors
max_length_inp = input_tensor.shape[1]
max_length_inp

188

In [27]:
input_tensor.shape

(86, 188)

In [28]:
# Creating training and validation sets using an 90-10 split
input_tensor_train, input_tensor_val = train_test_split(input_tensor, test_size=0.1)

# Show length
len(input_tensor_train), len(input_tensor_val)

(77, 9)

- Checking word to index mapping in the vocabulary

In [29]:
inp_lang.index_word[60]

'इस'

In [30]:
inp_lang.word_index['इस']

60

In [31]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [32]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[30])

Input Language; index to word mapping
1 ----> <start>
178 ----> चतुर
793 ----> चूहाएक
50 ----> चूहा
6 ----> था
5 ----> वह
206 ----> रास्ते
12 ----> पर
62 ----> जा
22 ----> रहा
6 ----> था
21 ----> उसे
329 ----> कपड़े
16 ----> का
3 ----> एक
132 ----> टुकड़ा
273 ----> मिल्रा
5 ----> वह
21 ----> उसे
330 ----> लेकर
207 ----> आगे
331 ----> बढ़ा
2 ----> <end>


In [33]:
input_tensor_train.shape

(77, 188)

- Create TensorFlow datasets for training and validation

In [34]:
BATCH_SIZE = 64
BUFFER_SIZE = len(input_tensor_train)

vocab_inp_size = len(inp_lang.word_index) + 1

dataset_train = tf.data.Dataset.from_tensor_slices(input_tensor_train).shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)

dataset_val = tf.data.Dataset.from_tensor_slices(input_tensor_val).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)

- Custom data generator that can be used for batching and shuffling data in training 
- Yield batches of data (of size batch_size) for training.

In [35]:
def generator(data, min_index, max_index, shuffle=False, batch_size=128):
    if max_index is None:
        max_index = len(data) - 1
    i = min_index
    
    while True:
        if i + batch_size >= max_index:
            i = min_index
        rows = np.arange(i, min(i + batch_size, max_index))
        i += len(rows)

        # preallocate
        samples = np.zeros((len(rows), data.shape[-1])) #1st dim:rows, 2nd dim:features
        
        # fill in: For each row selected, select the number of timesteps to sample from
        for j, row in enumerate(rows):                         # for each observation (row)
            indices = rows[j]
            samples[j] = data[indices]
            
        yield samples

In [36]:
BATCH_SIZE_P = 4
BUFFER_SIZE_P = len(input_tensor_train_p)

inp_gen_p = generator(
    input_tensor_train_p,
    min_index   = 0,
    max_index   = 39,
    batch_size  = BATCH_SIZE_P)

In [37]:
example_input_batch_p = next(inp_gen_p)
example_input_batch_p.shape

(4, 137)

- Preview index mappings of generated input batch

In [38]:
example_input_batch_p

array([[  3.,  43.,  26.,   2.,  27.,  14., 534., 153.,  14., 535.,  16.,
        536.,  27.,   2.,  32.,   7.,   2.,  29.,   4.,   1.,  22.,  12.,
        537.,   9., 538., 539.,   4., 110., 111.,  29.,   6., 163., 540.,
        541.,  95.,  61.,  29., 180., 542.,   2., 111., 543.,   4., 161.,
        122.,  56., 544.,  70.,  61.,   2.,  31., 545.,  10., 546.,  48.,
          8.,  29.,   9., 547., 548.,  10., 178.,   1.,  29.,  10., 549.,
          7.,  22., 172., 550.,  12., 101.,  29., 551., 552., 553.,  21.,
         25.,  36.,  20.,   2.,  89.,  29.,  21., 554., 555.,   4.,   1.,
         29., 184., 556.,  29.,   9.,  72., 557., 558.,   4.,   5.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  3.,   2., 142., 363.,  27.,   2., 143.,   4.,   2.,  31.,   1.,

In [39]:
for (batch, inp) in enumerate(dataset_train):
    print(inp)
    break


tf.Tensor(
[[  1   5 664 ...   0   0   0]
 [  1  14  28 ...   0   0   0]
 [  1 188 189 ...   0   0   0]
 ...
 [  1  58  20 ...   0   0   0]
 [  1  38  16 ...   0   0   0]
 [  1   3  36 ...   0   0   0]], shape=(64, 188), dtype=int32)


### Building an RNN model to generate vector embeddings

We create a RNN model for creating the vector embeddings for our Hindi text

- It starts with an embedding layer to convert words into dense vectors, followed by two LSTM layers to learn sequential patterns. 
- The output layer predicts the next word in the sequence, using a `softmax` activation. 
- The model uses the `Adam` optimizer and `sparse_categorical_crossentropy` loss for classification tasks.

In [40]:
def build_hindi_rnn_model(vocab_size, max_sequence_length, embedding_dim = 100, rnn_units = 256):
    # Input layer with fixed sequence length
    inputs = Input(shape=(max_sequence_length,))
    
    # Embedding layer
    embedding = Embedding(vocab_size, embedding_dim)(inputs)
    
    # LSTM layers
    lstm1 = LSTM(rnn_units, return_sequences=True)(embedding)
    lstm2 = LSTM(rnn_units)(lstm1)
    
    # Output layer to predict next word
    outputs = Dense(vocab_size, activation='softmax')(lstm2)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer   = 'adam',
        loss        = 'sparse_categorical_crossentropy',
        metrics     = ['accuracy']
    )
    
    return model

- To prepare the training data for the RNN model, we create a sliding windows of words. 
- Each window of `window_size` words is used as input X, and the next word in the sequence is the target y. 
- Sequences shorter than window_size + 1 are ignored, and padding is applied to maintain consistent window size.

In [41]:
def prepare_training_data(sequences, window_size=5):
    X, y = [], []
    
    for sequence in sequences:
        # Skip sequences shorter than window_size + 1
        if len(sequence) < window_size + 1:
            continue
            
        # Filter out padding (zeros)
        sequence = [x for x in sequence if x != 0]
        
        for i in range(len(sequence) - window_size):
            window = sequence[i:i + window_size]
            target = sequence[i + window_size]
            
            # Pad window if necessary
            if len(window) < window_size:
                window = window + [0] * (window_size - len(window))
            
            X.append(window)
            y.append(target)
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

- Train the model with 90-10 size for validation data with batch size of 64

In [42]:
def train_model(model, X, y, validation_split=0.1, epochs=50, batch_size=64):
    try:
        history = model.fit(
            X, 
            y,
            validation_split = validation_split,
            epochs           = epochs,
            batch_size       = batch_size,
            verbose          = 1
        )
        return history
    except Exception as e:
        # Debugging
        print(f"Error during training: {str(e)}")
        print(f"Input shape: {X.shape}")
        print(f"Target shape: {y.shape}")
        raise

- Return the weights of that layer, which correspond to the word embeddings

In [43]:
def get_embeddings(model):
    return model.get_layer(index = 1).get_weights()[0]

- Return the vector corresponding to the given word index

In [44]:
def get_word_embedding(model, word_idx):
    embeddings = get_embeddings(model)
    return embeddings[word_idx]

- Finds the top `k` most similar words to a given word, based on cosine similarity between their embeddings
- Compare the word's embedding with all other embeddings and return the `k` words with the highest similarity
- Return the words if tokenizer is available, else return the indices

In [45]:
def find_similar_words(model, word_idx, top_k=5, tokenizer=None):
    embeddings = get_embeddings(model)
    word_embedding = embeddings[word_idx]
    
    # Find similar words by using cosine similarity
    similarities = np.dot(embeddings, word_embedding) / (
        np.linalg.norm(embeddings, axis=1) * np.linalg.norm(word_embedding)
    )
    
    # Get top k similar words
    top_indices = np.argsort(similarities)[-top_k-1:][::-1][1:]
    
    if tokenizer:
        return [(tokenizer.index_word[idx], similarities[idx]) for idx in top_indices]
    
    return [(idx, similarities[idx]) for idx in top_indices]

### Train the RNN model

In [46]:
def train_hindi_rnn_embeddings(input_tensor_train, vocab_inp_size, inp_lang, max_sequence_length):
    
    # Initialize the model with fixed sequence length
    model = build_hindi_rnn_model(
        vocab_size          = vocab_inp_size,
        max_sequence_length = max_sequence_length,
        embedding_dim       = 100,
        rnn_units           = 256
    )
    
    # Prepare training data
    X_train, y_train = prepare_training_data(input_tensor_train)
    
    # Train the model
    print("Starting training...")
    history = train_model(model, X_train, y_train, epochs=50)

    # Save the model
    model.save("models/Part2.h5")
    
    return model

- Train, save, and load RNN model on the input data. 
- Retrieve the embedding for the word "कहानी", and also find and print the most similar words to "कहानी" based on cosine similarity

In [47]:
max_sequence_length = 5

# Create and train the model
rnn_model = train_hindi_rnn_embeddings(
    input_tensor_train,
    vocab_inp_size,
    inp_lang,
    max_sequence_length
)

model = load_model('models/Part2.h5')

word = 'कहानी'
if word in inp_lang.word_index:
    word_idx = inp_lang.word_index[word]

    # Find embeddings for the word
    embedding = get_word_embedding(model, word_idx)
    print(f"\nEmbedding vector for '{word}':", embedding[:5])

    # Find other similar words
    similar_words = find_similar_words(model, word_idx, tokenizer=inp_lang)
    print(f"\nWords similar to '{word}':")
    
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity:.4f}")

Starting training...
Epoch 1/50
36/36 [==============================] - 5s 24ms/step - loss: 6.6810 - accuracy: 0.0214 - val_loss: 6.2923 - val_accuracy: 0.0472
Epoch 2/50
36/36 [==============================] - 0s 10ms/step - loss: 6.1304 - accuracy: 0.0192 - val_loss: 6.5288 - val_accuracy: 0.0039
Epoch 3/50
36/36 [==============================] - 0s 8ms/step - loss: 5.9904 - accuracy: 0.0262 - val_loss: 6.7623 - val_accuracy: 0.0472
Epoch 4/50
36/36 [==============================] - 0s 7ms/step - loss: 5.9320 - accuracy: 0.0236 - val_loss: 6.9386 - val_accuracy: 0.0472
Epoch 5/50
36/36 [==============================] - 0s 7ms/step - loss: 5.8838 - accuracy: 0.0284 - val_loss: 6.9717 - val_accuracy: 0.0079
Epoch 6/50
36/36 [==============================] - 0s 7ms/step - loss: 5.8441 - accuracy: 0.0306 - val_loss: 6.9422 - val_accuracy: 0.0512
Epoch 7/50
36/36 [==============================] - 0s 7ms/step - loss: 5.8061 - accuracy: 0.0332 - val_loss: 6.9722 - val_accuracy: 0.01

### Save the word embeddings to a text file

- Iterate over the vocabulary in the tokenizer, retrieve the embedding for each word, and write the word along with its embedding vector to the file

In [48]:
def save_embeddings_to_file(model, tokenizer, file_name='hindi_word_embeddings.txt'):
    with open(file_name, 'w', encoding='utf-8') as f:
        
        # Iterate over each word and its index in the tokenizer vocabulary
        for word, idx in tokenizer.word_index.items():
            embedding = get_word_embedding(model, idx)
            if embedding is not None:
                # Convert the embedding vector to a string
                embedding_str = ' '.join(map(str, embedding))
                f.write(f"{word} {embedding_str}\n")
    
    print(f"Embeddings saved to {file_name}")

save_embeddings_to_file(model, inp_lang)

Embeddings saved to hindi_word_embeddings.txt
